In [1]:
##### load rainfall data and locations
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import os
D = np.genfromtxt('/Users/Claire/Desktop/graphical_model/HW2c/data/data.txt',delimiter=None,dtype = int)
loc = np.genfromtxt('/Users/Claire/Desktop/graphical_model/HW2c/data/locations.txt',delimiter=None)
m,n = D.shape
print (m,n)

2760 30


In [2]:
#Compute the empirical mutual information score for each pair of locations
px = np.zeros((n,2))
for i in range(n):
  for j in range(m):
    px[i][D[j][i]] +=1
  px[i] /=px[i].sum()
  
pxx = np.zeros((n,n,2,2))
for i in range(n):
  for j in range(n):
    for k in range(m):
      pxx[i][j][D[k][i]][D[k][j]] +=1
    pxx[i][j] /=m

I = np.zeros((n,n))
for i in range(n):
  for j in range(n):
    for k in range(2):
      for q in range(2):
        I[i][j] +=pxx[i][j][k][q] * np.log(pxx[i][j][k][q]/(px[i][k]*px[j][q]))


/Users/Claire/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:20: RuntimeWarning: divide by zero encountered in log
/Users/Claire/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:20: RuntimeWarning: invalid value encountered in double_scalars


In [3]:
edges = np.genfromtxt('/Users/Claire/Desktop/graphical_model/HW2c/data/edges.txt', delimiter=None, dtype=int)

G = nx.Graph()
for u,v in edges:
  if I[u][v] > 0:
    G.add_edge(u,v,weight=-1*I[u][v], color = 'b')
    
T = nx.minimum_spanning_tree(G)

print(T.edges())

location = {i:loc[i] for i in range(n)}
colors = [G[u][v]['color'] for u,v in T.edges()]
nx.draw_networkx(T, pos=location, edge_color=colors)
#plt.show()
print ('end')


[(0, 2), (1, 4), (2, 17), (2, 16), (3, 12), (3, 29), (3, 5), (4, 6), (5, 6), (7, 10), (7, 13), (8, 13), (9, 10), (10, 14), (11, 12), (11, 14), (13, 15), (17, 18), (17, 20), (18, 19), (20, 27), (21, 24), (21, 25), (21, 27), (22, 26), (22, 28), (22, 23), (27, 29), (28, 29)]
end


In [4]:
L = 0.0
for i in range(n):
    L =L + px[i][0] * np.log(px[i][0]) + px[i][1] * np.log(px[i][1])
    
for u,v in T.edges():
    L = L + I[u][v]

print (L)

-11.0986143278


In [5]:
T_loopy = nx.Graph()
for i in range(n):
    T_loopy.add_node(i)
for u,v in edges:
    T_loopy.add_edge(u,v,color='b')
colors = [T_loopy[u][v]['color'] for u,v in T_loopy.edges()]
nx.draw_networkx(T_loopy, pos=location, edge_color=colors)
#plt.show()
print ('end')

end


In [6]:
os.chdir('/Users/Claire/Desktop/graphical_model/HW2c')
import pyGM as gm
X = [gm.Var(i,2) for i in range(n)]
margin = [gm.Factor([X[u],X[v]],1.0) for u,v in edges]
i = 0
for u,v in edges:
    margin[i].table = pxx[u][v]
#    print (margin[i].table)
    i = i + 1
print ('end')

end


In [18]:
factors = [gm.Factor([X[u],X[v]],1.0) for u,v in edges]
for i in range(len(factors)):
    factors[i].table = np.random.rand(2,2)
    
iteration = 20

LL = np.zeros(iteration)
model = gm.GraphModel(factors)

for i in range(iteration):
    for j in range(len(edges)):
        u,v = edges[j]
        model_ve = gm.GraphModel(factors)
        pri = [1.0 for xi in X]
        pri[u],pri[v] = 2.0,2.0
        order = gm.eliminationOrder(model_ve, orderMethod='minfill', priority=pri)[0]
        sumElim = lambda F,Xlist: F.sum(Xlist)   # helper function for eliminate
        model_ve.eliminate(order[:-2], sumElim)  # eliminate all but last two
        puv= model_ve.joint()
        lnZ = np.log(puv.sum())
        puv /= puv.sum()
        factors[j].table = factors[j].table * margin[j].table / puv.table
    model = gm.GraphModel(factors)
    LL[i] = np.sum(model.logValue(d) for d in D)/m - lnZ
    print (i, LL[i])
plt.plot(np.arange(0,iteration), LL, marker = 'o')
plt.show()
print ('end')


0 -10.8513816379
1 -9.84558468609
2 -9.72858546141
3 -9.69914558037
4 -9.69306861303
5 -9.69172028546
6 -9.69145497724
7 -9.69139742605
8 -9.69138277553
9 -9.69137901583
10 -9.69137807993
11 -9.69137785779
12 -9.69137780958
13 -9.69137780009
14 -9.69137779824
15 -9.69137779781
16 -9.69137779769
17 -9.69137779766
18 -9.69137779765
19 -9.69137779765
end
